# Guide to Converting Seurat Objects to HDF5/loom Files

### Example Filepaths Can Be Changed Here

In [1]:
in_path <- "/gpfs/data/nneretti/data/datasets/scRNA_seq/exampleSeuratBrainOld3and4.rds"
out_path <- "/gpfs/data/nneretti/data/datasets/scRNA_seq/example1_out.loom"

# Saving Using hdf5r #

## Setting up the Environment ##

We will be using [Seurat](https://satijalab.org/seurat/), and [hdf5r](https://cran.r-project.org/web/packages/hdf5r/vignettes/hdf5r.html)

In order to use hdf5, make sure you have the hdf5 library loaded:

        > module load hdf5/1.10.5 
This repository has an environment.yml file which can be used to easily 
install all of the packages necessary to a new Anaconda environment as described in
[the official Conda documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file):

        > conda env create -f environment.yml

This new environment should already be ready to run jupyter notebooks due to the *jupyterlab* and *r-irkernel* dependencies in the environment.yml file. However, if launching jupyter using

        > /users/*username*/anaconda/seurat4-env/bin/jupyter-lab 
        
Doesn't result in the new R kernel showing up as an option, you may have to additionally to register the new R kernel from within the new R prompt:

        > /users/*username*/anaconda/seurat4-env/bin/R
        > install.packages('IRkernel')
        > IRkernel::installspec()

Now any jupyter instance run from 

        > /users/*username*/anaconda/seurat4-env/bin/jupyter-lab 
        
should have the proper R kernel available. 

To start, we load the Seurat and hdf5r libraries:

In [2]:
library(Seurat)
library(hdf5r)

Attaching SeuratObject



## Loading
Then we load the Seurat object:

In [3]:
seurat_obj <- readRDS(in_path)
DefaultAssay(object = seurat_obj) <- "RNA"
seurat_obj

An object of class Seurat 
35838 features across 1234 samples within 3 assays 
Active assay: RNA (17594 features, 2000 variable features)
 2 other assays present: SCT, integrated
 2 dimensional reductions calculated: pca, umap

## Saving
### Creating connection to new hdf5 file

In [4]:
file.h5 <- H5File$new(out_path, mode = "w")
file.h5

Class: H5File
Filename: /gpfs/data/nneretti/data/datasets/scRNA_seq/example1_out.loom
Access type: H5F_ACC_RDWR

# Groups
We will create groups as described in the [loom hdf5 specification](http://linnarssonlab.org/loompy/format/). 
All datasets except for the raw counts at "/matrix" will be contained within one of these groups. 
The specification explains what each group means, but the ones we are mostly concerned with are:
1. "layers", which contains all matrices of size num_cells x num_genes,
2. "row_attrs" which for us contains all metadata datasets describing individual cells,
3. "col_attrs", which for us contains all metadata describing individual genes,

We also added the group "scaled" which is not included in the loom specification as a place to store scaled matrices which have fewer genes - the number of genes is reduced to a cutoff of the top $n$ variable genes.

In [5]:
layers.grp <- file.h5$create_group("layers")
row_attrs.grp <- file.h5$create_group("row_attrs")
col_attrs.grp <- file.h5$create_group("col_attrs")
attrs.grp <- file.h5$create_group("attrs")
attrs.grp[["LOOM_SPEC_VERSION"]] <- "3.0.0"
col_graphs.grp <- file.h5$create_group("col_graphs")
row_graphs.grp <- file.h5$create_group("row_graphs")
scaled.grp <- file.h5$create_group("scaled")

## Raw Counts ##
Raw RNA counts are stored in the object[["RNA"]]@counts slot - the counts slot for the RNA assay. 
#### /matrix ####
Every loom file should have a root level /matrix dataset in which we will put the raw counts.

In [6]:
head(as.data.frame(seurat_obj[["RNA"]]@counts))

,AAACCTGGTCCTCCAT-4,AAACGGGAGCTCCCAG-4,AAAGCAACAAGTTGTC-4,AAAGCAAGTCACACGC-4,AAAGCAAGTGTAATGA-4,AAAGCAATCCTCAACC-4,AAAGTAGTCGCCCTTA-4,AAATGCCGTATATCCG-4,AACCATGAGGTGATTA-4,AACCATGGTCTGGAGA-4,⋯,GGCAATTGTTCCGTCT-4,GGGTTGCGTTAGAACA-4,GGTGCGTAGCTAGGCA-4,GTACGTAAGCAAATCA-4,GTAGTCATCAACACCA-4,GTATCTTTCGTGGACC-4,TGGTTAGAGGCATGGT-4,TTATGCTTCTATCCTA-4,TTCTCAAGTACACCGC-4,TTTCCTCCACCAGGTC-4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Xkr4,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Rp1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Sox17,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Mrpl15,0,1,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Lypla1,0,0,0,0,0,1,0,0,0,0,⋯,0,0,0,0,1,0,0,0,0,0
Gm37988,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [7]:
file.h5[["matrix"]] <- (as.matrix(seurat_obj[["RNA"]]@counts))

In [8]:
file.h5[["matrix"]]

Class: H5D
Dataset: /matrix
Filename: /gpfs/data/nneretti/data/datasets/scRNA_seq/example1_out.loom
Access type: H5F_ACC_RDWR
Datatype: H5T_IEEE_F64LE
Space: Type=Simple     Dims=1234 x 17594     Maxdims=Inf x Inf
Chunk: 32 x 32

## Column (Gene) Metadata
Now is a good time to add the Column (gene) metadata - right now we only have the gene names,
but you could include other information about the genes such as accession numbers,
or whether a gene was included in a variable features search.

In [9]:
col_attrs.grp[["Name"]] <- row.names(seurat_obj[["RNA"]]@counts)
col_attrs.grp[["Name"]]

Class: H5D
Dataset: /col_attrs/Name
Filename: /gpfs/data/nneretti/data/datasets/scRNA_seq/example1_out.loom
Access type: H5F_ACC_RDWR
Datatype: H5T_STRING {
      STRSIZE H5T_VARIABLE;
      STRPAD H5T_STR_NULLTERM;
      CSET H5T_CSET_ASCII;
      CTYPE H5T_C_S1;
   }
Space: Type=Simple     Dims=17594     Maxdims=Inf
Chunk: 1024

## Row (Cell) Metadata
Next we will add the cell level (here row level) metadata; 
this is stored in the meta.data slot of the Seurat object.

In [10]:
head(seurat_obj@meta.data)

,orig.ident,percent.mito,MID,res.1,res.0.9,res.0.8,res.1.2,nCount_RNA,nFeature_RNA,old.ident,percent.ribo,percent.mt,nCount_SCT,nFeature_SCT
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>
AAACCTGGTCCTCCAT-4,24 months,0.03018035,Old Mice 3 and 4,16,10,11,16,2717,1186,Microglia,26.831064,3.018035,3393,1164
AAACGGGAGCTCCCAG-4,24 months,0.02638738,Old Mice 3 and 4,3,3,3,3,7352,1931,Microglia,17.913493,2.638738,4271,1760
AAAGCAACAAGTTGTC-4,24 months,0.03122731,Old Mice 3 and 4,4,4,4,4,1377,706,Microglia,12.563544,3.122731,3348,813
AAAGCAAGTCACACGC-4,24 months,0.02841596,Old Mice 3 and 4,2,2,2,2,3308,1357,Microglia,8.434099,2.841596,3603,1355
AAAGCAAGTGTAATGA-4,24 months,0.02370773,Old Mice 3 and 4,3,3,3,3,7719,1819,Microglia,22.438140,2.370773,4151,1580
AAAGCAATCCTCAACC-4,24 months,0.01813785,Old Mice 3 and 4,4,4,4,4,3308,1324,Microglia,19.891173,1.813785,3615,1324


In [11]:
row_attrs.grp[["Barcode"]] <- row.names(seurat_obj@meta.data)


In [12]:
for (i in names(seurat_obj@meta.data)){
    row_attrs.grp[[i]] <- seurat_obj@meta.data[[i]]
}
row_attrs.grp

Class: H5Group
Filename: /gpfs/data/nneretti/data/datasets/scRNA_seq/example1_out.loom
Group: /row_attrs
Listing:
         name    obj_type dataset.dims dataset.type_class
      Barcode H5I_DATASET         1234         H5T_STRING
          MID H5I_DATASET         1234         H5T_STRING
   nCount_RNA H5I_DATASET         1234          H5T_FLOAT
   nCount_SCT H5I_DATASET         1234          H5T_FLOAT
 nFeature_RNA H5I_DATASET         1234        H5T_INTEGER
 nFeature_SCT H5I_DATASET         1234        H5T_INTEGER
    old.ident H5I_DATASET         1234         H5T_STRING
   orig.ident H5I_DATASET         1234         H5T_STRING
 percent.mito H5I_DATASET         1234          H5T_FLOAT
   percent.mt H5I_DATASET         1234          H5T_FLOAT
< Printed 10, out of 15>

## Normalized Data ##
Normalized counts are stored in the object[["RNA"]]@data slot - the data slot for the RNA assay. 

We will put this as the "normalized" dataset in the "layers" group.

In [13]:
head(as.data.frame(seurat_obj[["RNA"]]@data))

,AAACCTGGTCCTCCAT-4,AAACGGGAGCTCCCAG-4,AAAGCAACAAGTTGTC-4,AAAGCAAGTCACACGC-4,AAAGCAAGTGTAATGA-4,AAAGCAATCCTCAACC-4,AAAGTAGTCGCCCTTA-4,AAATGCCGTATATCCG-4,AACCATGAGGTGATTA-4,AACCATGGTCTGGAGA-4,⋯,GGCAATTGTTCCGTCT-4,GGGTTGCGTTAGAACA-4,GGTGCGTAGCTAGGCA-4,GTACGTAAGCAAATCA-4,GTAGTCATCAACACCA-4,GTATCTTTCGTGGACC-4,TGGTTAGAGGCATGGT-4,TTATGCTTCTATCCTA-4,TTCTCAAGTACACCGC-4,TTTCCTCCACCAGGTC-4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Xkr4,0,0.0000000,0,0,0,0.000000,0,0,0,0,⋯,0,0,0,0,0.000000,0,0,0,0,0
Rp1,0,0.0000000,0,0,0,0.000000,0,0,0,0,⋯,0,0,0,0,0.000000,0,0,0,0,0
Sox17,0,0.0000000,0,0,0,0.000000,0,0,0,0,⋯,0,0,0,0,0.000000,0,0,0,0,0
Mrpl15,0,0.8587354,0,0,0,1.392022,0,0,0,0,⋯,0,0,0,0,0.000000,0,0,0,0,0
Lypla1,0,0.0000000,0,0,0,1.392022,0,0,0,0,⋯,0,0,0,0,0.556717,0,0,0,0,0
Gm37988,0,0.0000000,0,0,0,0.000000,0,0,0,0,⋯,0,0,0,0,0.000000,0,0,0,0,0


In [14]:
layers.grp[["normalized"]] <- (as.matrix(seurat_obj[["RNA"]]@data))
layers.grp[["normalized"]]

Class: H5D
Dataset: /layers/normalized
Filename: /gpfs/data/nneretti/data/datasets/scRNA_seq/example1_out.loom
Access type: H5F_ACC_RDWR
Datatype: H5T_IEEE_F64LE
Space: Type=Simple     Dims=1234 x 17594     Maxdims=Inf x Inf
Chunk: 32 x 32

## Scaled + Normalized Data ##
Scaled and Normalized counts are stored in the object[["RNA"]]@scale.data slot - the scale.data slot for the RNA assay. 
We will store these in the "matrix" dataset of the "scaled" group,
and store the variable gene names in the "vargenes" dataset of the "scaled" group. 

In [15]:
head(as.data.frame(seurat_obj[["RNA"]]@scale.data))

,AAACCTGGTCCTCCAT-4,AAACGGGAGCTCCCAG-4,AAAGCAACAAGTTGTC-4,AAAGCAAGTCACACGC-4,AAAGCAAGTGTAATGA-4,AAAGCAATCCTCAACC-4,AAAGTAGTCGCCCTTA-4,AAATGCCGTATATCCG-4,AACCATGAGGTGATTA-4,AACCATGGTCTGGAGA-4,⋯,GGCAATTGTTCCGTCT-4,GGGTTGCGTTAGAACA-4,GGTGCGTAGCTAGGCA-4,GTACGTAAGCAAATCA-4,GTAGTCATCAACACCA-4,GTATCTTTCGTGGACC-4,TGGTTAGAGGCATGGT-4,TTATGCTTCTATCCTA-4,TTCTCAAGTACACCGC-4,TTTCCTCCACCAGGTC-4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Sox17,-0.389809577,-0.336413151,-0.31468884,-0.28843408,-0.3568345251,-0.3350830110,-0.35596199,-0.343434941,-0.299520367,-0.3310835445,⋯,-0.33548897,-0.30087460,-0.4381966,-0.4230042,-0.29413115,-0.4308949,-0.4007203,-0.4230684,-0.4458257,-0.26901129
Npbwr1,-0.002233966,-0.005402154,-0.01435629,-0.01477763,0.0007693487,0.0043119376,-0.07607157,-0.005192167,-0.006065610,-0.0001527243,⋯,-0.07797839,-0.04663312,-0.1395931,-0.1341855,-0.02106932,-0.1373699,-0.1116083,-0.1319541,-0.1541303,-0.02523886
3110035E14Rik,0.147990633,-0.039682333,-0.16910414,-0.25351425,0.0664306105,0.0234412165,-0.46825976,-0.016433373,-0.158788388,-0.0198824453,⋯,-0.54500794,5.01260323,-0.6527782,-0.6624969,-0.27938925,-0.6600509,3.2258918,-0.6468493,-0.7297414,-0.38622613
Prex2,0.029215329,-0.124821274,-0.25490867,-0.32065233,-0.0222868227,-0.0425255842,-0.70019031,-0.106382461,-0.217831183,-0.0912441953,⋯,-0.76533749,-0.57143619,-1.0645696,-1.0529064,-0.36252810,-1.0622692,-0.9069873,-1.0331287,-1.1744023,-0.45855680
A830018L16Rik,0.017593924,0.014270840,-0.03370455,-0.02842609,0.0457099725,0.0737541271,-0.42141487,0.013450952,0.021253806,0.0477922762,⋯,-0.42689862,-0.22590283,-0.8323450,-0.7949160,5.48578428,-0.8166417,10.0000000,10.0000000,-0.9230463,-0.08625575
Msc,-0.010550252,-0.011088526,-0.01930060,-0.01838021,-0.0057106497,-0.0008899801,-0.08579361,-0.011233220,-0.009869655,-0.0053385027,⋯,-0.08672169,-0.05224277,10.0000000,-0.1498658,-0.02523610,-0.1535951,-0.1251686,-0.1475422,-0.1718458,-0.02828304


In [16]:
scaled.grp[["matrix"]] <- (as.matrix(seurat_obj[["RNA"]]@scale.data))
scaled.grp[["vargenes"]] <- row.names(seurat_obj[["RNA"]]@scale.data)
scaled.grp

Class: H5Group
Filename: /gpfs/data/nneretti/data/datasets/scRNA_seq/example1_out.loom
Group: /scaled
Listing:
     name    obj_type dataset.dims dataset.type_class
   matrix H5I_DATASET  1234 x 2000          H5T_FLOAT
 vargenes H5I_DATASET         2000         H5T_STRING

# Finished
We have now created the HDF5 File. All that's left is to close the file connection, to free it up to be read by other processes.

In [17]:
file.h5$close_all()

## Miscellaneous Notes ##
* We are generally following the [loom hdf5 specification](http://linnarssonlab.org/loompy/format/) for potentially better interoperability and use of convenience functions such as those found in 
[loompy](http://loompy.org/), [SCopeLoomR](https://github.com/aertslab/SCopeLoomR), and [SCENIC](https://pyscenic.readthedocs.io/en/latest/). 
* The loom specification is flexible and does not specify whether to use columns as genes or as cells.  In the official tutorial, they use genes as rows and cells as columns. However, hdf5 is stored in row-major, which should make row access faster than column access. In machine learning applications, we generally want to take batches as slices of a few rows at a time, so we will be using cells as rows and genes as columns.
    *  In addition, the way we created datasets in hdf5r conveniently automatically chooses datatypes and chunk sizes, but these can be tweaked to increase performance, as described in the [tutorial](https://cran.r-project.org/web/packages/hdf5r/vignettes/hdf5r.html), [the package documentation](https://cran.r-project.org/web/packages/hdf5r/hdf5r.pdf), and [StackOverflow](https://stackoverflow.com/questions/48385256/optimal-hdf5-dataset-chunk-shape-for-reading-rows).